In [1]:
from web3 import Web3, HTTPProvider

In [2]:
# load abi and bytecode
with open('../contracts/auction/abi.json', 'r') as abi_file:
    abi = abi_file.read()
with open('../contracts/auction/bytecode', 'r') as bc:
    bytecode = bc.read()

In [3]:
# web3.py instance
w3 = Web3(HTTPProvider('http://localhost:8545'))

In [4]:
# account data
owner_account_address = w3.toChecksumAddress('0xdc88f96bcbedbd92539fefe2c91148cb67d272e4')
guest_account_address = w3.toChecksumAddress('0x0635c434ec5f88f7dc8e44e0b878116d71d2a658')
owner_account_password = 'password'
guest_account_password = 'second_password'

In [5]:
# prepare contract object
auction_deploy = w3.eth.contract(abi=abi, bytecode=bytecode)

In [6]:
# get address balance before auction
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

777154549000001001 wei
0.777154549000001001 ether


In [7]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)
# Submit the transaction which deploys the contract with biddingTime 300 seconds (5 minutes)
tx_hash = auction_deploy.constructor(300).transact({
    'from': owner_account_address
})
print(tx_hash.hex())

0xdd87108179d55e886820e7b8eb5075513338deeee7f7805b9af0a4ec31729c67


In [8]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt)

AttributeDict({'blockHash': HexBytes('0x34ea577001a8d7df6db30af4bf7bba8476f0530f6c11fa3e24ef12920d26cf40'), 'blockNumber': 5813249, 'contractAddress': '0x698dB522b8810A9C0f85c1359BC23Ce48012dbd1', 'cumulativeGasUsed': 7072391, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 452494, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': None, 'transactionHash': HexBytes('0xdd87108179d55e886820e7b8eb5075513338deeee7f7805b9af0a4ec31729c67'), 'transactionIndex': 22}

In [9]:
# Create the contract instance with the newly-deployed address (can't use without address)
contract = w3.eth.contract(
    address = w3.toChecksumAddress(tx_receipt['contractAddress']),
    abi=abi,
)

In [10]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [11]:
# prepare contract function object
bid_obj = contract.functions.bid()
bid_gas_estimated = bid_obj.estimateGas({
    'from': owner_account_address,
    'value': Web3.toWei(0.03, 'ether')
})
print(w3.eth.gasPrice, 'current gas price in Wei, testnet')
print(bid_gas_estimated, 'estimated for the transaction')
transaction_cost = float(Web3.fromWei(w3.eth.gasPrice * bid_gas_estimated, 'ether')) * 132.45
print(transaction_cost, 'payed for the transactions in USD')

1000000000 current gas price in Wei, testnet
63651 estimated for the transaction
0.00843057495 payed for the transactions in USD


In [12]:
# 1 ether 132,45$ | 3.97$ == 0.03 == 30 000 000 000 000 000 wei
bid_tx = bid_obj.transact({
    'from': owner_account_address,
    'value': Web3.toWei(0.03, 'ether'),
    'gas': bid_gas_estimated,
    'gas_price': w3.eth.gasPrice
})
print(bid_tx.hex())

0xe49efa1abfa3053118f058450d03524a4e23294269ae96377673d008f99dc3fa


In [13]:
# Wait for the transaction to be mined, and get the transaction receipt
print(w3.eth.waitForTransactionReceipt(bid_tx))

AttributeDict({'blockHash': HexBytes('0xcafc653e3f6bd2e29faa9bb5c7286ab7357dda6762e70686c8094ca666836a65'), 'blockNumber': 5813252, 'contractAddress': None, 'cumulativeGasUsed': 6814540, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 63651, 'logs': [AttributeDict({'address': '0x698dB522b8810A9C0f85c1359BC23Ce48012dbd1', 'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf0809674f6aab8ad300')], 'data': '0x000000000000000000000000dc88f96bcbedbd92539fefe2c91148cb67d272e4000000000000000000000000000000000000000000000000006a94d74f430000', 'blockNumber': 5813252, 'transactionHash': HexBytes('0xe49efa1abfa3053118f058450d03524a4e23294269ae96377673d008f99dc3fa'), 'transactionIndex': 10, 'blockHash': HexBytes('0xcafc653e3f6bd2e29faa9bb5c7286ab7357dda6762e70686c8094ca666836a65'), 'logIndex': 20, 'removed': False})], 'logsBloom': HexBytes('0x0000000000000000000000000080000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [14]:
# get highestBidder using call()
print(contract.functions.highestBidder().call(), 'highestBidder')
# get highestBid
print(contract.functions.highestBid().call(), 'highestBid')

0xdC88f96BCBEdbD92539Fefe2c91148Cb67d272E4 highestBidder
30000000000000000 highestBid


In [15]:
# make another bid with other account, bid is less then previous
w3.personal.unlockAccount(guest_account_address, guest_account_password)

True

In [16]:
# 1 ether 132,45$ | 5.30$ == 0.040 == 40 000 000 000 000 000 wei
bid_tx = bid_obj.transact({
    'from': guest_account_address,
    'value': Web3.toWei(0.040, 'ether'),
})
print(w3.eth.waitForTransactionReceipt(bid_tx))

AttributeDict({'blockHash': HexBytes('0x5e9f8b97d89624ba6095911c190374c0cd53727624a5273142acfced15837820'), 'blockNumber': 5813253, 'contractAddress': None, 'cumulativeGasUsed': 7885649, 'from': '0x0635c434ec5f88f7dc8e44e0b878116d71d2a658', 'gasUsed': 54416, 'logs': [AttributeDict({'address': '0x698dB522b8810A9C0f85c1359BC23Ce48012dbd1', 'topics': [HexBytes('0xf4757a49b326036464bec6fe419a4ae38c8a02ce3e68bf0809674f6aab8ad300')], 'data': '0x0000000000000000000000000635c434ec5f88f7dc8e44e0b878116d71d2a658000000000000000000000000000000000000000000000000008e1bc9bf040000', 'blockNumber': 5813253, 'transactionHash': HexBytes('0xab9ffb6c1f8fefbd640585cee8ccc09109fb7ffeb15437587e6f907f1532156d'), 'transactionIndex': 36, 'blockHash': HexBytes('0x5e9f8b97d89624ba6095911c190374c0cd53727624a5273142acfced15837820'), 'logIndex': 23, 'removed': False})], 'logsBloom': HexBytes('0x0000000000000000000000000080000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [17]:
# get highestBidder using call()
print(contract.functions.highestBidder().call(), 'highestBidder')
# get highestBid
print(contract.functions.highestBid().call(), 'highestBid')

0x0635c434EC5F88F7Dc8E44E0B878116d71D2A658 highestBidder
40000000000000000 highestBid


In [18]:
# get address balance before withdraw
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

746638404000001001 wei
0.746638404000001001 ether


In [19]:
# unlock account before transaction
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [20]:
# withdraw bid, owner
withdraw_obj = contract.functions.withdraw()
withdraw_tx = withdraw_obj.transact({
    'from': owner_account_address,
})
print(withdraw_tx.hex())

0xc95a4cf81c730eed12fbb5879762f5421c7e9fa6932ecdc500e3c3cb70497059


In [21]:
# Wait for the transaction to be mined, and get the transaction receipt
print(w3.eth.waitForTransactionReceipt(withdraw_tx))

AttributeDict({'blockHash': HexBytes('0x640789939532f04852c1a1ac7445d08e0d4cdb5dc3dce12559b89e80f7fc48e9'), 'blockNumber': 5813256, 'contractAddress': None, 'cumulativeGasUsed': 6644198, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 19410, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 1, 'to': '0x698db522b8810a9c0f85c1359bc23ce48012dbd1', 'transactionHash': HexBytes('0xc95a4cf81c730eed12fbb5879762f5421c7e9fa6932ecdc500e3c3cb70497059'), 'transactionIndex': 15})

In [22]:
# get address balance after withdraw
begin_owner_balance = w3.eth.getBalance(owner_account_address)
print(begin_owner_balance, 'wei')
print(Web3.fromWei(begin_owner_balance, 'ether'), 'ether')

776618994000001001 wei
0.776618994000001001 ether


In [23]:
# get auctionEndTime using call()
auction_end_time = contract.functions.auctionEndTime().call()
print(auction_end_time)

1560793659


In [24]:
# get beneficiary address
print(contract.functions.beneficiary().call())

0xdC88f96BCBEdbD92539Fefe2c91148Cb67d272E4


In [27]:
# unlock account to transact
w3.personal.unlockAccount(owner_account_address, owner_account_password)

True

In [28]:
# finish auction if now >= auctionEndTime, transfer data of highestBid to auction owner
auction_end_obj = contract.functions.auctionEnd()
auction_end_tx = auction_end_obj.transact({
    'from': owner_account_address,
})
print(w3.eth.waitForTransactionReceipt(auction_end_tx.hex()))

AttributeDict({'blockHash': HexBytes('0x8903dcf4cbafabacfa4b0e05b0e54fb6ebd5846cd2868b50baee0f68e5d4ab9d'), 'blockNumber': 5813294, 'contractAddress': None, 'cumulativeGasUsed': 1652730, 'from': '0xdc88f96bcbedbd92539fefe2c91148cb67d272e4', 'gasUsed': 51919, 'logs': [AttributeDict({'address': '0x698dB522b8810A9C0f85c1359BC23Ce48012dbd1', 'topics': [HexBytes('0xdaec4582d5d9595688c8c98545fdd1c696d41c6aeaeb636737e84ed2f5c00eda')], 'data': '0x0000000000000000000000000635c434ec5f88f7dc8e44e0b878116d71d2a658000000000000000000000000000000000000000000000000008e1bc9bf040000', 'blockNumber': 5813294, 'transactionHash': HexBytes('0xe360fc5cb88282ff311bc5c3daa4315d8859680857974615884f48d6fddaa3ea'), 'transactionIndex': 43, 'blockHash': HexBytes('0x8903dcf4cbafabacfa4b0e05b0e54fb6ebd5846cd2868b50baee0f68e5d4ab9d'), 'logIndex': 8, 'removed': False})], 'logsBloom': HexBytes('0x00000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [29]:
# get address balance after auction
owner_balance = w3.eth.getBalance(owner_account_address)
print(Web3.fromWei(begin_owner_balance, 'ether'), 'begin balance ether')
print(Web3.fromWei(owner_balance, 'ether'), 'end balance ether')

0.776618994000001001 begin balance ether
0.816567075000001001 end balance ether
